In [ ]:
!pip install langchain fastapi uvicorn pydantic transformers huggingface_hub

In [ ]:
!pip install langchain-community


In [7]:
!pip install uvicorn


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# LangChain Agent with Gemma 2 (Small)

In [ ]:
# mood_agent_ollama.py

from fastapi import FastAPI, Request
from pydantic import BaseModel
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

app = FastAPI()

# LangChain Ollama LLM setup
llm = Ollama(model="gemma:2b")  # Ensure you've run: ollama pull gemma:2b

# Prompt template for mood-boosting quote
template = """You are a compassionate AI that provides short, powerful, and uplifting quotes.
Mood detected: {mood}

Respond with one short quote that would uplift someone who is feeling this way.
Quote:"""
prompt = PromptTemplate.from_template(template)

@app.post("/webhook")
async def receive_mood(request: Request):
    data = await request.json()
    mood = data.get("mood", "").strip().lower()

    if not mood:
        return {"error": "No mood provided"}

    final_prompt = prompt.format(mood=mood)
    quote = llm(final_prompt).strip()

    # Display quote on screen or terminal
    print(f"\n🧠 Mood: {mood}\n💬 Quote: {quote}\n")

    return {"quote": quote}


In [8]:
!uvicorn mood_agent:app --host 0.0.0.0 --port 8000

ERROR:    Error loading ASGI app. Could not import module "mood_agent".


In [ ]:
!curl -X POST http://localhost:8000/webhook \
     -H "Content-Type: application/json" \
     -d '{"mood": "sad"}'